# Sistema de Votación por Tramos para Eurovisión 2025

En este cuaderno se implementa un sistema de votación alternativo para Eurovisión basado en tramos de posiciones. En lugar de asignar puntos únicamente a los 10 primeros puestos, se otorgan puntos a los países según el bloque de posiciones en el que hayan quedado, siguiendo la siguiente escala:

- Puesto 1: **12 puntos**
- Puesto 2: **10 puntos**
- Puesto 3: **8 puntos**
- Puestos 4 y 5: **7 puntos**
- Puestos 6 y 7: **6 puntos**
- Puestos 8 y 9: **5 puntos**
- Puestos 10, 11 y 12: **4 puntos**
- Puestos 13, 14, 15 y 16: **3 puntos**
- Puestos 17, 18, 19 y 20: **2 puntos**
- Puestos 21, 22, 23 y 24: **1 punto**
- Puestos 25 y 26: **0 puntos**

Este sistema permite valorar de forma más amplia la posición obtenida por cada país, otorgando puntos a más participantes y diferenciando mejor los resultados intermedios.

Además, se han generado los archivos `tabla_jurado_tramos.xlsx`, `tabla_publico_tramos.xlsx` y `tabla_total_tramos.xlsx` en formato Excel, disponibles para el análisis, así como los datos crudos en `data_cleaned.json`.


Idea de Berme Rubio ([x.com/bermerubio](https://x.com/bermerubio))

## Extracción de datos

El archivo `data_cleaned.json` contiene los votos otorgados por cada país, tanto del jurado como del televoto, en formato estructurado. Estos datos serán la base para aplicar el sistema de puntuación por tramos.

In [12]:
import json

with open('data_cleaned.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    print("Leídos", len(data), "registros.")

Leídos 37 registros.


In [13]:
voting_countries = data.keys()
participating_countries = [d['country'] for d in data['Croatia']['jury']]
non_participating_countries = set(voting_countries) - set(participating_countries)
print("Países votantes:", len(voting_countries))
print("Países participantes:", len(participating_countries))
print("Países votantes que no llegaron a la final:", len(non_participating_countries))

Países votantes: 37
Países participantes: 26
Países votantes que no llegaron a la final: 11


## Verificación de datos

El siguiente código comprueba que la suma de las posiciones asignadas por cada país es correcta, asegurando que no haya errores en los datos: los países que participan deben sumar 325 posiciones y los no participantes, 351.

In [14]:
for country in participating_countries:
    for type in data[country]:
        suma = sum(d['rank'] for d in data[country][type])
        if suma != 325:
            print("Error en el país", country, "tipo", type, "suma", suma)

In [15]:
for country in non_participating_countries:
    for type in data[country]:
        suma = sum(d['rank'] for d in data[country][type])
        if suma != 351:
            print("Error en el país", country, "tipo", type, "suma", suma)

## Mapeo de puntos

La siguiente función transforma la posición obtenida por cada país en puntos, aplicando el sistema de tramos definido al inicio del cuaderno.

In [16]:
def rank_to_points(rank):
    if rank == 1:
        return 12
    elif rank == 2:
        return 10
    elif rank == 3:
        return 8
    elif rank in (4, 5):
        return 7
    elif rank in (6, 7):
        return 6
    elif rank in (8, 9):
        return 5
    elif rank in (10, 11, 12):
        return 4
    elif rank in (13, 14, 15, 16):
        return 3
    elif rank in (17, 18, 19, 20):
        return 2
    elif rank in (21, 22, 23, 24):
        return 1
    else:
        return 0

## Obtención de tablas de puntos para jurado y público

Primero se define la función que genera la tabla de posiciones para cada país según los votos recibidos. Después, se aplicará el sistema de tramos para obtener las tablas de puntos tanto del jurado como del televoto.

In [17]:
import pandas as pd

def create_vote_table(vote_type):
    table = {}
    for voting_country, votes in data.items():
        for vote in votes[vote_type]:
            voted_country = vote['country']
            rank = vote['rank']
            if voted_country not in table:
                table[voted_country] = {}
            table[voted_country][voting_country] = rank
    df = pd.DataFrame(table).T
    df = df.sort_index().sort_index(axis=1)
    return df

### Obtención de la tabla de puntos de jurado y guardado en Excel

Se calcula la tabla de puntos asignados por el jurado a cada país aplicando el sistema de tramos, y se guarda en un archivo Excel para su posterior análisis.

In [18]:
df_jury = create_vote_table('jury')
df_jury_points = df_jury.map(rank_to_points)
total_jury_points = df_jury_points.sum(axis=1)
total_jury_points.sort_values(ascending=False, inplace=True)
df_jury_points['Total'] = total_jury_points
df_jury_points.to_excel('tabla_jurado_tramos.xlsx')

### Obtención de la tabla de puntos de televoto y guardado en Excel

Se calcula la tabla de puntos asignados por el televoto a cada país aplicando el sistema de tramos, y se guarda en un archivo Excel para su posterior análisis.

In [19]:
df_public = create_vote_table('public')
df_public_points = df_public.map(rank_to_points)
total_public_points = df_public_points.sum(axis=1)
total_public_points.sort_values(ascending=False, inplace=True)
df_public_points['Total'] = total_public_points
df_public_points.to_excel('tabla_publico_tramos.xlsx')

### Obtención de la tabla de puntos totales y guardado en Excel

Se suman los puntos obtenidos por cada país en el jurado y en el televoto para obtener la clasificación final según el sistema de tramos. La tabla resultante se guarda en un archivo Excel para su análisis. Se muestran los 10 primeros y los 10 últimos.

In [20]:
total_points = df_public_points + df_jury_points
total_points.sort_values('Total', ascending=False, inplace=True)
total_points.to_excel('tabla_total_tramos.xlsx')

In [ ]:
# Primeras diez posiciones
total_points.head(10)

,Albania,Armenia,Australia,Austria,Azerbaijan,Belgium,Croatia,Cyprus,Czechia,Denmark,...,Portugal,San Marino,Serbia,Slovenia,Spain,Sweden,Switzerland,Ukraine,United Kingdom,Total
Austria,14,15,12,0,13,17,16,14,7,11,...,14,9,18,20,13,16,14,11,12,505
Estonia,8,17,12,13,15,15,18,10,9,10,...,9,14,17,14,8,12,11,11,12,450
Israel,13,4,13,11,24,15,11,16,13,13,...,13,12,9,11,14,16,15,9,13,450
Sweden,11,14,15,12,9,13,13,9,10,19,...,10,5,9,13,10,0,16,9,11,430
Italy,13,7,4,13,13,10,19,9,8,11,...,20,17,7,24,10,7,20,15,5,399
France,16,22,6,10,5,11,6,15,10,5,...,10,9,15,8,12,9,14,6,6,367
Greece,19,11,19,4,4,10,9,24,7,7,...,6,16,15,5,5,8,11,3,9,350
Netherlands,9,10,5,10,8,11,6,12,9,4,...,11,7,7,9,14,13,11,6,4,334
Finland,4,14,15,17,8,10,11,6,7,13,...,5,5,8,7,9,17,8,6,12,333
Albania,0,6,9,9,10,9,12,8,6,5,...,11,11,7,11,9,12,13,8,7,332


In [ ]:
# Últimas 10 posiciones
total_points.tail(10)

,Albania,Armenia,Australia,Austria,Azerbaijan,Belgium,Croatia,Cyprus,Czechia,Denmark,...,Portugal,San Marino,Serbia,Slovenia,Spain,Sweden,Switzerland,Ukraine,United Kingdom,Total
Norway,6,8,3,4,8,3,10,7,12,6,...,4,5,6,10,6,9,4,13,4,230
Malta,2,10,14,11,7,2,2,5,7,5,...,2,5,6,2,8,4,2,5,11,206
Armenia,4,0,2,5,0,3,4,8,6,5,...,4,3,7,2,5,4,1,5,5,204
United Kingdom,3,4,5,8,5,2,2,2,10,8,...,6,6,4,2,8,1,7,11,0,187
Luxembourg,11,3,5,6,3,4,2,5,3,5,...,5,3,5,6,3,5,3,7,8,183
Iceland,4,2,7,7,3,3,5,4,6,13,...,3,5,3,6,3,8,5,4,5,177
Spain,11,4,5,3,6,4,6,5,2,3,...,9,5,6,2,0,7,6,1,5,174
Denmark,4,3,9,10,3,5,2,2,5,0,...,2,4,6,1,5,4,2,2,13,167
Portugal,5,6,0,2,9,2,3,5,6,2,...,0,7,2,6,4,2,9,9,4,166
San Marino,10,3,0,3,4,1,2,4,1,0,...,0,0,3,1,1,2,4,1,0,110
